<a href="https://colab.research.google.com/github/otwn/Geospatial-Analysis-Examples/blob/master/Geocoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import subprocess

# GeoPandas
try:
    import geopandas as gpd
except ImportError:
    print('geopandas package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geopandas'])
    import geopandas as gpd


In [ ]:
# Geemap
try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Google Colab
try: 
  import google.colab
  import geemap.eefolium as emap
except:
  import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()    

# Geocoder
* OpenStreetMap Nominatim geocoder

In [ ]:
from geopandas.tools import geocode

In [ ]:
result = geocode("Narita Airport", provider="nominatim")
result

/usr/local/lib/python3.6/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


,geometry,address
0,POINT (140.39331 35.77587),"成田国際空港, 久住停車場十余三線, 吉岡, 成田市, 千葉県, 286-0101, 日本 ..."


In [ ]:
point = result.geometry.iloc[0]
print("Latitude", point.y)
print("Longitude", point.x)

Latitude 35.77587145
Longitude 140.3933101399336


In [ ]:
result.address.iloc[0]

'成田国際空港, 久住停車場十余三線, 吉岡, 成田市, 千葉県, 286-0101, 日本 (Japan)'

In [ ]:
m1 = emap.Map(center=[35, 140], zoom=8)
emap.folium.Marker([point.y, point.x], popup=result.address.iloc[0]).add_to(m1)
m1

# In case you have only location name
## For example, you know top universities list

In [ ]:
import pandas as pd

universities = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Geospatial/top_universities.csv")
universities.info

<bound method DataFrame.info of                                   Name
0                 University of Oxford
1              University of Cambridge
2              Imperial College London
3                           ETH Zurich
4                                  UCL
..                                 ...
95                  Aalborg University
96  Paris Diderot University – Paris 7
97                University of Bergen
98   Lomonosov Moscow State University
99               University of Antwerp

[100 rows x 1 columns]>

In [ ]:
import numpy as np
def my_geocoder(row):
  try:
    point = geocode(row, provider="nominatim").geometry.iloc[0]
    return pd.Series({"Latitude":point.y, "Longitude":point.x, "geometry": point})
  except:
    return None

universities[ ["Latitude", "Longitude", "geometry"] ] = universities.apply(lambda x: my_geocoder(x["Name"]), axis=1)

universities = universities.loc[~np.isnan(universities["Latitude"])]

universities = gpd.GeoDataFrame(universities, geometry=universities.geometry)
universities.crs = {"init":"epsg:4326"}
universities.head()

/usr/local/lib/python3.6/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning
/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,Name,Latitude,Longitude,geometry
0,University of Oxford,51.758708,-1.255668,POINT (-1.25567 51.75871)
1,University of Cambridge,52.199852,0.119739,POINT (0.11974 52.19985)
2,Imperial College London,51.498871,-0.175608,POINT (-0.17561 51.49887)
3,ETH Zurich,47.377327,8.547509,POINT (8.54751 47.37733)
4,UCL,51.524126,-0.132930,POINT (-0.13293 51.52413)


In [ ]:
m2 = emap.Map(center=[54,15], zoom=4)

for idx, row in universities.iterrows():
  emap.folium.Marker([row["Latitude"], row["Longitude"]], popup=row["Name"]).add_to(m2)

m2